In [ ]:
# define a new class for documents

class document:

    def __init__(self, ID) -> None:
        self.id = int(ID)
        self.title = ""
        self.abstract = ""
        self.pub_date = 0
        self.author = list()
        self.info = ""
        self.ref = list()

    def set_title(self, title):
        self.title = title
    
    def set_abstract(self, abstract):
        self.abstract += abstract

    def set_pubdate(self, pubdate):
        self.pubdate = int(pubdate)

    def set_author(self, author):
        self.author.append(author)

    def set_info(self, info):
        self.info += info

    def set_ref(self, ref):
        self.ref.append(ref)

    def get_id(self):
        return self.id

    def get_title(self):
        return self.title

    def get_abstract(self):
        return self.abstract

    def get_pubdate(self):
        return self.pub_date

    def get_author(self):
        return self.author

    def get_info(self):
        return self.info

    def get_ref(self):
        return self.ref
    

In [ ]:
docs = list()
IDs = list()                #.I
titles = list()             #.T
abstracts = list()          #.W
pub_dates = list()          #.B
authors = list()            #.A
info = list()               #.N
refrences = list()          #.X

In [23]:
#read CISI.ALL file and fill lists

with open("CISI.ALL", encoding = "latin-1") as all_files:
    current_doc = None
    for line in all_files.readlines():
        if line.startswith(".I"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 0
            b = 0
            id = line[3]
            IDs.append(id)
            current_doc = document(id)
            docs.append(current_doc)
        elif line.startswith(".T"):
            t = 1
            a = 0
            w = 0
            x = 0
            n = 0
            b = 0
        elif line.startswith(".W"):
            t = 0
            a = 0
            w = 1
            x = 0
            n = 0
            b = 0
        elif line.startswith(".B"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 0
            b = 1
        elif line.startswith(".N"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 1
            b = 0
        elif line.startswith(".X"):
            t = 0
            a = 0
            w = 0
            x = 1
            n = 0
            b = 0
        elif line.startswith(".A"):
            t = 0
            a = 1
            w = 0
            x = 0
            n = 0
            b = 0
        elif t == 1:
            # title
            titles.append(line)
            current_doc.set_title(line)
            t = 0
        elif a == 1:
            # author
            authors.append(line)
            current_doc.set_author(line)
        elif b == 1:
            # publication date
            pub_dates.append(line)
            current_doc.set_pubdate(line)
        elif w == 1:
            # abstract
            abstracts.append(line)
            current_doc.set_abstract(line)
        elif x == 1:
            # references
            refrences.append(line)
            current_doc.set_ref(line)
        elif n == 1:
            # info
            info.append(line)
            current_doc.set_info(line)
        else:
            continue


In [ ]:
# normalization
